<a href="https://colab.research.google.com/github/reza-latifi/Iris-Classification-ANN/blob/main/IrisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy import where
from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from collections import defaultdict

# Load Dataset

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
names = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class"])

In [ ]:
df.head(7)

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa


# Preprocess

In [ ]:
# Delete missing values
df = df.dropna()
df.shape

(150, 5)

In [ ]:
# One-Hot encode Strings

labelencoder = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(enc.fit_transform(labelencoder.fit_transform(df['Class']).reshape(-1, 1)).toarray())
df = df.join(enc_df)

In [ ]:
df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class,0,1,2
0,5.1,3.5,1.4,0.2,Iris-setosa,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,Iris-setosa,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,Iris-setosa,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,Iris-setosa,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,Iris-setosa,1.0,0.0,0.0


# Seperate Train, Test, and Validation sets

In [ ]:
classes = set(df['Class'])
classes

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [ ]:
IrisSetosa = df.loc[where(df['Class'] == 'Iris-setosa')]
IrisVersicolor = df.loc[where(df['Class'] == 'Iris-versicolor')]
IrisVirginica = df.loc[where(df['Class'] == 'Iris-virginica')]

train = pd.concat((IrisSetosa[:35], IrisVersicolor[:35], IrisVirginica[:35])) # 35 samples for Train set
test = pd.concat((IrisSetosa[35:45], IrisVersicolor[35:45], IrisVirginica[35:45])) # 10 samples for Test set
validation = pd.concat((IrisSetosa[45:], IrisVersicolor[45:], IrisVirginica[45:])) # 5 samples for Validaiton set

train_x = train.values[:,0:4]
train_y = train.values[:,5:]

test_x = test.values[:,0:4]
test_y = test.values[:,5:]

validation_x = validation.values[:,0:4]
validation_y = validation.values[:,5:]

In [ ]:
# Remove extra field
df.drop('Class', axis=1, inplace=True)

# ANN

In [ ]:
# Parameters
epochs = 100
hiddens = np.array([5, 3]) # Actually it is (4, 5, 3)
layers = 3 # length of (4, 5, 3)
batch = 5

ALPHA = 0.5
BETA = 0.9
ETA = 0.05


In [ ]:
def sigmoid2(x):
  x = x.astype(float)
  return 1 / (1 + np.exp(-1*x))**2

In [ ]:
def sigmoid2_gradient(x):
  x = x.astype(float)
  return 2 * np.exp(-1*x) / (np.exp(-1*x) + 1)**3

In [ ]:
def train(inputs, lable, weights = None):
    global hiddens, layers

    if weights is None:
      # Initialize weights
        weights = {}
        for w in range(layers): # For each layer
            if w == 0:
                weights[w] = np.random.random((inputs.shape[1], hiddens[w])) # (4, 5)
            elif w == len(hiddens + 1):
                weights[w] = np.random.random((hiddens[w - 1], lable.shape[1])) # (5, 3)
            else:
                weights[w] = np.random.random((hiddens[w - 1], hiddens[w])) # (3, 1)
    
    for epoch in range(100): # For each epoch
        print(f"Epoch: {epoch+1}")
        prev = {}
        
        sampleCounter = 0
        error = defaultdict(int)
        for i in range(inputs.shape[0]): # For all 105 train samples
            sampleCounter += 1
            s = {} # Output of each layer
            delta = {}

            activaionInput = {}
            # Forward propagation
            sample = inputs[i]
            for layer in range(layers): # Move sample through layers
                net = np.dot(sample, weights[layer]) # Compute input of next layer
                s[layer] = ALPHA*net # Multiply next layer inputs to Alpha
                activaionInput[layer] = sigmoid2(s[layer]) # Compute activation function of the next layer inputs and give it to the next layer
                sample = activaionInput[layer] # Select output of current layer as input of the next layer

            # Backward propagation
            for layer in reversed(range(layers)):
                if layer == len(hiddens + 1):
                    error[layer] += (lable[i] - activaionInput[layer]) * sigmoid2_gradient(s[layer]) # error * gradient for the last layer
                else:
                    error[layer] += np.dot(error[layer + 1], np.transpose(weights[layer + 1])) * sigmoid2_gradient(s[layer]) # error * gradient for the last layer
                prev[layer] = 0

            # Update parameters
            if sampleCounter % 5 == 0:
              print(f"        Batch: {sampleCounter/5} =>", end=" ")
              for layer in range(layers):
                  if layer == 0:
                      delta[layer] = ETA * np.dot(error[layer][:,None], inputs[i][:,None].T) + BETA * prev[layer]
                  else:
                      delta[layer] = ETA * np.dot(error[layer][:,None], activaionInput[layer - 1][:,None].T) + BETA * prev[layer]

                  prev[layer] = delta[layer]
                  weights[layer] = weights[layer] + delta[layer].T
              print(f"Parameters updated")
              error = defaultdict(int)
    return weights

In [ ]:
def test(inputs, lable, weights, classes):
    classes = tuple(classes)
    hyp = []
    real = []
    miss = 0
    for i in range(inputs.shape[0]): # For each test sample
        sample = inputs[i]
        for j in range(len(weights)): # For each layer
            # Calculate output
            net = np.dot(sample, weights[j])
            s = ALPHA * net
            out = sigmoid2(s)
            sample = out
        guess = classes[np.argmax(out)]
        realClass = classes[np.argmax(lable[i])]
        if guess != realClass:
          miss += 1
        hyp.append(guess)
        real.append(realClass)
    print(accuracy_score(hyp, real))
    print(classification_report(hyp, real, zero_division=0))
    print(f"Total test samples: {len(inputs)}")
    print(f"Miss Classifications: {miss}")

In [ ]:
weights = train(train_x, train_y)

Epoch: 1
        Batch: 1.0 => Parameters updated
        Batch: 2.0 => Parameters updated
        Batch: 3.0 => Parameters updated
        Batch: 4.0 => Parameters updated
        Batch: 5.0 => Parameters updated
        Batch: 6.0 => Parameters updated
        Batch: 7.0 => Parameters updated
        Batch: 8.0 => Parameters updated
        Batch: 9.0 => Parameters updated
        Batch: 10.0 => Parameters updated
        Batch: 11.0 => Parameters updated
        Batch: 12.0 => Parameters updated
        Batch: 13.0 => Parameters updated
        Batch: 14.0 => Parameters updated
        Batch: 15.0 => Parameters updated
        Batch: 16.0 => Parameters updated
        Batch: 17.0 => Parameters updated
        Batch: 18.0 => Parameters updated
        Batch: 19.0 => Parameters updated
        Batch: 20.0 => Parameters updated
        Batch: 21.0 => Parameters updated
Epoch: 2
        Batch: 1.0 => Parameters updated
        Batch: 2.0 => Parameters updated
        Batch: 3.0 => Param

In [ ]:
test(test_x, test_y, weights, classes)

0.6666666666666666
                 precision    recall  f1-score   support

    Iris-setosa       1.00      0.50      0.67        20
Iris-versicolor       1.00      1.00      1.00        10
 Iris-virginica       0.00      0.00      0.00         0

       accuracy                           0.67        30
      macro avg       0.67      0.50      0.56        30
   weighted avg       1.00      0.67      0.78        30

Total test samples: 30
Miss Classifications: 10
